# bindPredict21

In [64]:
import sys
BASE_DIR = '..'
BINDPREDICT_DIR = '/afs/csail.mit.edu/u/s/samsl/Work/Applications/bindPredict'

sys.path.append(BINDPREDICT_DIR)
sys.path.append(BASE_DIR)

In [2]:
from config import FileSetter, FileManager
from bindEmbed21DL import BindEmbed21DL
from pathlib import Path

In [11]:
import pandas as pd
df = pd.read_csv(f'{BASE_DIR}/dataset/BindingDB/train.csv')

In [20]:
seqDict = {i:s for i,s in enumerate(sorted(df['Target Sequence'].unique()))}

In [25]:
with open('tmp.fasta','w+') as f:
    for k,v in seqDict.items():
        f.write(f">seq{k}\n{v}\n")

In [26]:
prediction_folder = f'{BASE_DIR}/saved_embeddings'
model_prefix = f'{BINDPREDICT_DIR}/trained_models/checkpoint'

In [28]:
query_fasta = 'tmp.fasta'
query_sequences = FileManager.read_fasta(query_fasta)
query_ids = list(query_sequences.keys())
ri = False  # Whether to write RI or Probabilities

In [42]:
import torch
from architectures import CNN2Layers
from src.prot_feats import ProtBert_f

In [72]:
pbf = ProtBert_f(pool=False)

In [129]:
md = CNN2Layers(1024,128,5,1,2,0)
md.load_state_dict(torch.load(f"{model_prefix}5.pt", map_location='cuda:1')['state_dict'])
md = md.eval()
cnn_first = md.conv1[:2]

In [130]:
def seq_2_bindPredict_embedding(seq):
    with torch.set_grad_enabled(False):
        protbert_e = pbf(seq)
        bindpredict_e = cnn_first(protbert_e.view(1,1024,-1))
        return bindpredict_e.mean(axis=2).squeeze()

In [131]:
seq_2_bindPredict_embedding('MVMAMTVAGCCCGGGGGGGAMMV')

tensor([-0.0128,  0.0051, -0.0617,  0.1206,  0.1205,  0.0114, -0.0163,  0.0984,
        -0.0282, -0.1139,  0.0068,  0.1120, -0.0165, -0.0552,  0.1050, -0.0455,
        -0.0951, -0.0745,  0.0772,  0.1247, -0.0314,  0.1897, -0.0330,  0.1080,
        -0.0796, -0.0348,  0.0873,  0.1461, -0.0031, -0.1554,  0.1273, -0.0142,
         0.0501,  0.0058, -0.0128, -0.0100,  0.0440,  0.0944, -0.1034,  0.1109,
        -0.0360,  0.0584, -0.0574, -0.0132, -0.0346,  0.0643, -0.0318,  0.0139,
        -0.1004, -0.0223, -0.0225, -0.1330, -0.0497,  0.0685,  0.0078, -0.0094,
        -0.0889,  0.0402, -0.0894, -0.0671,  0.0781,  0.0050, -0.0349, -0.0260,
        -0.0267,  0.0245,  0.0925,  0.0424,  0.0908, -0.0189, -0.0460,  0.0248,
         0.0904,  0.0391,  0.0387,  0.0797, -0.0264,  0.0162,  0.0255,  0.0664,
        -0.0605, -0.0143,  0.0273,  0.0529, -0.0665,  0.0919,  0.0033,  0.0175,
        -0.0323,  0.0633, -0.0384, -0.1396, -0.0045,  0.1506,  0.0398,  0.1056,
        -0.0298,  0.2399,  0.1226,  0.10

# NetSurf-P3.0

In [128]:
import sys
BASE_DIR = '..'
NETSURF_DIR = '/afs/csail.mit.edu/u/s/samsl/Work/Applications/bindPredict'

sys.path.append(NETSURF_DIR)
sys.path.append(BASE_DIR)